# UK Sales: Variance from ZEV Targets

Based on concepts initially found in the document: ZEV Mandate 2, from related documents.

Requires nbformat>=4.2.0 to display plotly plots.

Created for New AutoMotive by Corrin Reilly

## Imports

In [1]:
# Packages
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Modules
from modules.connector import MyBigQuery, MySQL

## Initialise Connections

In [2]:
# BigQuery Connection.
bq = MyBigQuery(
    credentials_file="./credentials/New AutoMotive Index-487e031dc242.json"
)

# MySQL Connection.
sql = MySQL(
    db="ecc",
    credentials_files="./credentials/vrn-database_credentials.json",
    GCR=False
)

Connected to ecc database at 35.242.185.231!
Deployment in GCR: False



## Constants

In [3]:
# Maps

## Map of standard New Automotive colours for plots.
COLOR_MAP = {
    "Diesel": "black",
    "Petrol": "#183363",
    "Other": "#808080",
    "Hybrid": "#244d8c",
    "Pure electric": "green"
}

## Map of month boundaries for each quarter.
QUARTER_MAP = {
    "Q1": {
        "first": "01",
        "last": "03"
    },
    "Q2": {
        "first": "04",
        "last": "06"
    },
    "Q3": {
        "first": "07",
        "last": "09"
    },
    "Q4": {
        "first": "10",
        "last": "12"
    }
}

# Global constants to reduce changes required for each run
# TODO:- Set each before any new run.

## Percentage value as a decimal.
MANDATE_VALUE = 0.22
## Year for BigQuery query.
QUERY_YEAR = 2023
## Year for ZEV Mandate target.
MANDATE_YEAR = 2024
## Quarter for BigQuery query.
QUARTER = "Q1"

## Collect Data

In [4]:
df = sql.from_sql_to_pandas(
    sql_query="SELECT make, fuelType, registrationNumber FROM dataOther WHERE monthOfFirstRegistration >= '{year}-{first}' AND monthOfFirstRegistration <= '{year}-{last}'".format(
        year=QUERY_YEAR, first=QUARTER_MAP[QUARTER]["first"], last=QUARTER_MAP[QUARTER]["last"]
    )
)
df

,make,fuelType,registrationNumber
0,AUDI,Diesel,1231
1,AUDI,Hybrid,475
2,AUDI,Petrol,4742
3,AUDI,Pure electric,1504
4,BMW,Diesel,231
...,...,...,...
278,VOLKSWAGEN,Petrol,10800
279,VOLKSWAGEN,Pure electric,2421
280,VOLVO,Hybrid,3556
281,VOLVO,Petrol,1


## Process Data

In [5]:
# Remove "other" manufacturer as it is a more miscellaneous category.
df = df[df["make"] != "Other"]

In [6]:
# List unique manufacturers.
makes = df["make"].unique()

# Group data set as we only need a breakdown of sales by fuelType for each make.
df_grouped = df.groupby(
                by=["make", "fuelType"]
            )["registrationNumber"].sum()
df_grouped

make        fuelType     
AUDI        Diesel            4094
            Hybrid            1657
            Petrol           16835
            Pure electric     5194
BMW         Diesel            1070
                             ...  
VOLKSWAGEN  Petrol           22878
            Pure electric     5029
VOLVO       Hybrid            7973
            Petrol               3
            Pure electric     2351
Name: registrationNumber, Length: 94, dtype: int64

In [7]:
# Array of values for the total EV sales in the quarter.
total_evs = []
# Array of values for the number of EV sales required to hit target percentage.
required_evs = []
# Array of values for the percentage from the zev mandate target.
pcnt_from_mandate = []

for make in makes:
    # If the manufacturer has no EV sales we still wish to have a value so default as 0.
    total_evs_for_make = 0
    
    if "Pure electric" in df_grouped.loc[make].index:
        total_evs_for_make = df_grouped.loc[(make, "Pure electric")]
    
    # Sum sales of all vehicle types.
    total_sales = df_grouped.loc[make].sum()
    # Based on current total sales how many should be EV - further analysis could review how many further sales it would require to hit mandate.
    required_evs_for_make = round(total_sales * MANDATE_VALUE)
    # % share of sales that are EVs - required mandate value (%) -> if value is 0 they have hit target, if < 0 then mandate has not been met.
    pcnt_from_make = 100 * (total_evs_for_make / total_sales) - (MANDATE_VALUE * 100)
    
    # Append values to arrays for DataFrame.
    total_evs.append(total_evs_for_make)
    required_evs.append(required_evs_for_make)
    pcnt_from_mandate.append(pcnt_from_make)

In [8]:
# Create a DataFrame from which to plot.
df_plot = pd.DataFrame([
    makes, total_evs, required_evs, pcnt_from_mandate
]).T

df_plot.columns = ["manufacturer", "total_evs", "required_evs", "pcnt_from_mandate"]

df_plot.set_index("manufacturer", inplace=True)

## Plot 1: Total EV Sales Vs Required EV Sales

In [9]:
# Sort the data set to tidy the plot.
df_plot_1 = df_plot.sort_values("total_evs")

# Set bars plots
bars = [
    go.Bar(
        x=df_plot_1.index,
        y=df_plot_1["total_evs"],
        name="Total EV Sales",
        marker_color=COLOR_MAP["Hybrid"],
        hovertemplate="Manufacturer: %{x},<br />Total EV Sales: %{y}<extra></extra>"
    ),
    go.Bar(
        x=df_plot_1.index,
        y=df_plot_1["required_evs"],
        name="Required EV Sales",
        marker_color=COLOR_MAP["Pure electric"],
        hovertemplate="Manufacturer: %{x},<br />Required EV Sales: %{y}<extra></extra>"
    )
]

layout = go.Layout(
    title="Total EV Sales Vs. *Required EV Sales ({} {})".format(QUARTER, QUERY_YEAR),
    xaxis=dict(
        title="Manufacturer",
        tickangle=40
    ),
    yaxis=dict(
        title="Sales of EVs"
    ),
    title_x=0.5,
)

fig_1 = go.Figure(bars, layout)

fig_1.add_annotation(
    text="*% share according to {} ZEV mandate".format(MANDATE_YEAR),
    showarrow=False,
    xref="paper",
    yref="paper",
    x=0,
    y=-0.47
)

fig_1.show()

## Plot 2: Percentage Difference From ZEV Mandate

In [10]:
# Sort the data set to tidy the plot.
df_plot_2 = df_plot.sort_values("pcnt_from_mandate")

fig_2 = px.bar(
    df_plot_2,
    x=df_plot_2.index,
    y="pcnt_from_mandate",
    color_discrete_sequence=[COLOR_MAP["Hybrid"]],
    labels={
        "pcnt_from_mandate": "% From ZEV Mandate",
        "manufacturer": "Manufacturer"
    },
    title="Percentage Difference from ZEV Mandate* ({} {})".format(QUARTER, QUERY_YEAR),
)

fig_2.update_layout(
    title_x=0.5,
)

fig_2.update_xaxes(
    tickangle=40
)

fig_2.add_annotation(
    text="*Based on {} target of {}%".format(MANDATE_YEAR, round(MANDATE_VALUE * 100)),
    showarrow=False,
    xref="paper",
    yref="paper",
    x=0,
    y=-0.47
)

fig_2.show()